In [ ]:
pwd

In [ ]:
from pathlib import *
import numpy as np
import pandas as pd
from eyepackage.eye_parse import *
from eyepackage.behave_parse import *
from eyepackage.behave_eye_converge import *

In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
subids=["ec105","ec106","ec107","ec108"]    
pathstring='/Volumes/Voss_Lab/ECOG/ecog/locationspace/ecog.eye/'
eyepath=Path(pathstring)

In [ ]:
masternames=get_eye_files(subids,eyepath)

In [ ]:
study_all=masternames[masternames['phase']=="a"]
refresh_all=masternames[masternames['phase']=="b"]
recog_all=masternames[masternames['phase']=="c"]

for sub in subids:
    refresh_sub=[]
    refresh_sub=refresh_all[refresh_all['subject']==sub]

In [ ]:
# mastereyedf=[]
sub='ec108'
eye_events=parse_eye_line(refresh_sub,pathstring)
#     mastereyedf.append(eyedf)
eyedf=events_to_df(eye_events)
eyearray=eventsdf_cleanup(eyedf)

In [ ]:
behavestring='/Volumes/Voss_Lab/ECOG/ecog/locationspace/ecog.behave/'
extra='recogarray.txt'
behaveobj=[behavestring+sub+extra]
behavepath=Path(behaveobj[0])
behavepath.exists()

In [ ]:
behavearray=read_behave_file(behavepath)
subdict_change_coords=change_behave_coords(subids)

In [ ]:
#apply coordinate change to behavioral data if True in subdict
if subdict_change_coords[sub]==True:
    behavearray=apply_adjust_pres_coords(behavearray)
    

In [ ]:
subdict_change_coords

In [ ]:
reftimesextra='refreshtimes.txt'
reftimesobj=[behavestring+sub+reftimesextra]
reftimespath=Path(reftimesobj[0])
reftimespath.exists()

reftimes=read_times_file(reftimespath)

In [ ]:
eyebehave=eye_behave_combo(eyearray,behavearray,reftimes)

In [ ]:
#adjust timing to object onset
eyebehave=remove_baseline_eye(eyebehave)

In [ ]:
# calculate distances for start and end eye locations
startdistarray=calculate_dist(eyebehave,x1='xstart',y1='ystart',name='start')
enddistarray=calculate_dist(eyebehave,'xend','yend','end')

In [ ]:
# start locations
eyebehave=loc_view(eyebehave,startdistarray,'startloc')
eyebehave=loc_view(eyebehave,enddistarray,'endloc')

In [ ]:
#append start & end distances to eyebehave array
eyebehave=pd.concat([eyebehave, startdistarray, enddistarray], axis=1)

In [ ]:
eyebehavedict=eyebehave.to_dict('records')
eyebehavedict=assign_screenview(eyebehavedict,'xstart','ystart','start')
eyebehavedict=assign_screenview(eyebehavedict,'xend','yend','end')

In [ ]:
new_previous_events=adjust_fix_before_blink(eyebehavedict)
corrected_eye_events=adjust_event_after_blink(new_previous_events)
subcleandf=eyedict_backto_df(corrected_eye_events)

In [ ]:
l = [i +'hey' for i in etypes]
l

In [ ]:
type(events)
events[1]

In [ ]:
from mypackage2.mymodule import a_function_i_wrote as f2

In [ ]:
f1()

In [ ]:
f2()

In [ ]:
ls

In [ ]:
cd mypackage2

In [ ]:
ls

In [6]:
%reload_ext autoreload
%autoreload 2

In [18]:
from eyepackage.runEyeBehaveAnalysis import run, set_behavior_path, set_times_path

In [19]:
sub = 'ec107'
behavestring='/Volumes/Voss_Lab/ECOG/ecog/locationspace/ecog.behave/'

bpath = set_behavior_path(sub, behavestring)
tpath = set_times_path(sub, behavestring)
bpath, tpath

(PosixPath('/Volumes/Voss_Lab/ECOG/ecog/locationspace/ecog.behave/ec107recogarray.txt'),
 PosixPath('/Volumes/Voss_Lab/ECOG/ecog/locationspace/ecog.behave/ec107refreshtimes.txt'))

In [13]:
def eye_behave_combo(eyearray,behavearray,timesarray):
    eyebehave=eyearray.copy()

    eyecols=eyebehave.columns.tolist()
    behavecols=['loc1x','loc1y','loc2x','loc2y','loc3x','loc3y','recog loc','same/diff','cond']
    allcols=eyecols+behavecols+['objonset','trialend']
    eyebehave=eyebehave.reindex(columns=allcols)

    for trial,col in enumerate(behavearray):
        eyetrialevents=(eyebehave['trialnum']==trial+1)
        eyetrial=eyebehave.loc[eyetrialevents]

        behtrialevents=(behavearray['refresh order']==trial+1)
        behtrial=behavearray.loc[behtrialevents]
        for col in behavecols:
            eyetrial.loc[eyetrialevents,col]=behtrial.iloc[0][col]

        objonsetmask=timesarray.index==trial
        onsettrial=timesarray.loc[objonsetmask]
        eyetrial.loc[eyetrialevents,'objonset']=onsettrial.iloc[0]['objonset']
        eyetrial.loc[eyetrialevents,'trialend']=onsettrial.iloc[0]['trialend']

        eyebehave.loc[eyetrialevents]=eyetrial
    return eyebehave


